In [4]:
import os
import pandas as pd
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import requests
import numpy as np

load_dotenv()
email = os.getenv("AZ_USER")
password = os.getenv("AZ_PASSWORD")
option = Options()
option.page_load_strategy = 'normal'


# Clean up file
with open("filtered_elements.html", "w") as f:
    pass

In [6]:
def to_float(x):
    try:
        return float(x)
    except ValueError:
        return np.nan


class AmazonScraper:
    def __init__(self, email, password, year, options):
        self.email = email
        self.password = password
        self.year = year
        self.driver = webdriver.Chrome(options=options)

    def save_screenshot(self, filename):
        self.driver.save_screenshot(filename)

    def wait_for(self, seconds):
        WebDriverWait(self.driver, seconds)

    def login(self):
        print("Starting login process")
        self.navigate_to_login_page()
        self.wait_for(2)
        print("Entering email")
        self.enter_email()
        self.save_screenshot("email_entered.png")
        self.click_continue()
        self.wait_for(3)
        print("Entering password")
        self.enter_password()
        self.save_screenshot("password_entered.png")
        print("Sign in")
        self.click_sign_in()
        input("wait for human to complete captcha")
        self.wait_for(10)
        self.save_screenshot("logged_in.png")
        self.wait_for(10)
        self.save_screenshot("loged_2.png")

    def get_orders(self):
        print("Starting to get orders")
        self.click_orders()
        self.wait_for(8)
        self.save_screenshot("order_page.png")
        print("Selecting year")
        self.click_year_select_filter()
        self.wait_for(5)
        self.save_screenshot("select_pressed.png")
        page = 0
        while True:
            page += 1
            print(f"scraping orders page {page}")
            self.save_ordered_products()
            self.wait_for(2)
            try:
                next_button = self.driver.find_element(
                    By.CSS_SELECTOR, '.a-last a')
                next_button.click()
            except NoSuchElementException:
                break

    def get_title(self, soup):
        try:
            # Outer Tag Object
            title = soup.find("span", attrs={"id": "productTitle"})

            # Inner NavigatableString Object
            title_value = title.text

            # Title as a string value
            title_string = title_value.strip()

        except AttributeError:
            title_string = ""

        return title_string

    def get_price(self, soup):
        try:
            price = soup.find(
                "span", attrs={"id": "priceblock_ourprice"}
            ).string.strip()
        except AttributeError:
            try:
                # If there is some deal price
                price = soup.find(
                    "span", attrs={"class": "a-offscreen"}).string.strip()
            except:
                price = ""

        return price

    def get_categories(self, soup, title):
        try:
            categories_div = soup.find(
                "div", attrs={"id": "wayfinding-breadcrumbs_container"}
            )
            categories = categories_div.find_all(
                "a", attrs={"class": "a-link-normal a-color-tertiary"}
            )
            category_list = [category.get_text(strip=True)
                             for category in categories]
        except AttributeError:
            print(f"Coudn't find categories for {title}")
            category_list = []
        return category_list

    def get_products_as_df(self) -> pd.DataFrame:
        with open("filtered_elements.html", "r") as f:
            html_content = f.read()

        HEADERS = {"User-Agent": "", "Accept-Language": "en-US, en;q=0.5"}

        soup = BeautifulSoup(html_content, "html.parser")
        product_links = soup.select("a.a-link-normal")
        amazon_base_url = "https://www.amazon.com"
        products = {"title": [], "price": [], "categories": []}
        for link in product_links:
            link_url = link["href"]
            product_title = link.string
            print(product_title)

            # Prepend Amazon base URL if not present
            if not link_url.startswith(amazon_base_url):
                link_url = amazon_base_url + link_url

            try:
                new_webpage = requests.get(link_url, headers=HEADERS)
                new_soup = BeautifulSoup(new_webpage.content, "html.parser")
                try:
                    products["title"].append(self.get_title(new_soup))
                    products["price"].append(self.get_price(new_soup))
                    products["categories"].append(
                        self.get_categories(new_soup, product_title))

                except NoSuchElementException:
                    print("Product not found. Skipping this product.")
                    continue
            except:
                print(f"Something went wrong when scraping - skipping this product")
                continue

        amazon_df = pd.DataFrame.from_dict(products)
        amazon_df["title"].replace("", np.nan, inplace=True)
        amazon_df = amazon_df.dropna(subset=["title"])
        amazon_df["price"] = amazon_df["price"].str.replace("$", "")
        amazon_df["price"] = amazon_df["price"].apply(to_float)
        amazon_df.dropna(subset=['price'], inplace=True)
        return amazon_df

    def navigate_to_login_page(self):
        self.driver.get(
            "https://www.amazon.com/ap/signin?openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.amazon.com%2F%3Fref_%3Dnav_custrec_signin&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=usflex&openid.mode=checkid_setup&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&"
        )

    def enter_email(self):
        email_input = self.driver.find_element(By.NAME, "email")
        email_input.send_keys(self.email)

    def click_continue(self):
        self.driver.find_element(By.ID, "continue").click()

    def enter_password(self):
        password_input = self.driver.find_element(By.NAME, "password")
        password_input.send_keys(self.password)

    def click_sign_in(self):
        self.wait_for(3)
        self.driver.find_element(By.ID, "signInSubmit").click()
        self.wait_for(3)

    def click_orders(self):
        self.driver.find_element(By.ID, "nav-orders").click()

    def click_year_select_filter(self):
        select_element = self.driver.find_element(By.ID, "time-filter")
        select = Select(select_element)
        select.select_by_visible_text(self.year)

    def save_ordered_products(self):
        print("Downloading Orders to file")
        elements = self.driver.find_elements(
            By.CSS_SELECTOR, "div.a-fixed-left-grid-col div.a-row a.a-link-normal"
        )
        html_contents = [element.get_attribute(
            "outerHTML") for element in elements]
        with open("filtered_elements.html", "a") as f:
            for html in html_contents:
                f.write(html + "\n")

    def close(self):
        self.driver.quit()



In [7]:
amazon = AmazonScraper(email, password, "2023", option)
amazon.login()
amazon.get_orders()
df = amazon.get_products_as_df()
df.to_csv("amazon_data.csv", header=True, index=False)

Starting login process
Entering email
Entering password
Sign in
Starting to get orders
Selecting year
scraping orders page 1
scraping orders page 2
scraping orders page 3
scraping orders page 4
scraping orders page 5
scraping orders page 6
scraping orders page 7
scraping orders page 8
scraping orders page 9

        Little Chicks Easy-Clean Potty Training Toilet Chair, Built in Splash Guard and Slip Resistant - Model CK055
    

        LINENSPA 8 Inch Memory Foam and Innerspring Hybrid Mattress – Twin Mattress – Bed in a Box – Medium Firm Mattress
    

        Logitech H390 Wired Headset for PC/Laptop, Stereo Headphones with Noise Cancelling Microphone, USB, In-Line Controls, Works with Chromebook - Black
    

        Earth's Best Organic Baby Food Pouches, Stage 2 Wholesome Breakfast Puree for Babies 6 Months and Older, Organic Banana Blueberry and Oat Puree, 3.5 oz Resealable Pouch (Pack of 12)
    

        【2-PACK】Magnetic phone holder for car, [ Super Strong Magnet ] [ with 4 M

In [9]:
%pip install jupysql duckdb-engine --quiet


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
%load_ext sql 

In [11]:
%sql duckdb://

In [12]:
%%sql
select * from amazon_data.csv limit 10


Running query in 'duckdb://'

title,price,categories
"Little Chicks Easy-Clean Potty Training Toilet Chair, Built in Splash Guard and Slip Resistant - Model CK055",18.99,"['Baby Products', 'Potty Training', 'Potties']"
LINENSPA 8 Inch Memory Foam and Innerspring Hybrid Mattress – Twin Mattress – Bed in a Box – Medium Firm Mattress,99.99,"['Home & Kitchen', 'Furniture', 'Bedroom Furniture', 'Mattresses & Box Springs', 'Mattresses']"
"Logitech H390 Wired Headset for PC/Laptop, Stereo Headphones with Noise Cancelling Microphone, USB, In-Line Controls, Works with Chromebook - Black",21.88,"['Electronics', 'Computers & Accessories', 'Computer Accessories & Peripherals', 'Audio & Video Accessories', 'Computer Headsets']"
"Earth's Best Organic Baby Food Pouches, Stage 2 Wholesome Breakfast Puree for Babies 6 Months and Older, Organic Banana Blueberry and Oat Puree, 3.5 oz Resealable Pouch (Pack of 12)",16.91,"['Baby Products', 'Feeding', 'Baby Foods', 'Meals']"
"【2-PACK】Magnetic phone holder for car, [ Super Strong Magnet ] [ with 4 Metal Plate ] iPhone Magnetic car mount for cell phone, [ 360° Rotation ] Universal Dashboard car Mount Fits All Smartphones",14.99,"['Cell Phones & Accessories', 'Accessories', 'Automobile Accessories', 'Cradles']"
"BRIOFOX Adhesive Shower Curtain Rod Holder | No Drilling | Invisible Wall Mount Holder | Transparent Rod Retainer, 2 Pack(Rod Not Included)",5.99,"['Home & Kitchen', 'Bath', 'Bathroom Accessories', 'Shower Curtains, Hooks & Liners', 'Shower Curtain Rods']"
Aviv Judaica Traditional Hanukkah LED Electric Menorah Powered by Battery or USB Chanukiah - 4' Micro USB Cable Included - Electronic Chanukah Candles Minorah Lighted Hanukah Decorations (Blue),23.99,"['Home & Kitchen', 'Home Décor Products', 'Candles & Holders', 'Candleholders', 'Menorahs']"
"HORIMOTE HOME Quilt Set King Size Cameo Rose, Classic Geometric Spots Stitched Pattern, Stone-Washed Microfiber Chic Rustic Look, Ultra Soft Lightweight Quilted Bedspread for All Season, 3 Pieces",52.99,"['Home & Kitchen', 'Bedding', 'Quilts & Sets', 'Quilt Sets']"
Gardeneer by Dalen HammerEdge Pound in Edging - 16 Durable Interlocking Pieces -18 feet of Coverage - Made in USA - Easy to Install - 1 Pack,34.85,"['Patio, Lawn & Garden', 'Gardening & Lawn Care', 'Plant Support Structures', 'Garden Edging']"
COSCOOL Shade Sail Hardware Kit for Sun Shade Sails Stainless Steel Installation Hardware Kit (5-inch Rectangular),11.99,"['Patio, Lawn & Garden', 'Patio Furniture & Accessories', 'Umbrellas & Shade', 'Shade Sail Hardware']"


In [24]:
%%sql
create or replace table amazon_data as (
select title,price,cast(categories as varchar[]) as categories from read_csv_auto('amazon_data.csv')
)



Running query in 'duckdb://'

Count
96


In [25]:
%%sql
select categories[1] as category, sum(price) as spend
from amazon_data 
group by 1 

Running query in 'duckdb://'

column_name,column_type,null,key,default,extra
title,VARCHAR,YES,None,None,None
price,DOUBLE,YES,None,None,None
categories,VARCHAR[],YES,None,None,None
